<a href="https://colab.research.google.com/github/sharafinajwa/Face-Tone-Classification/blob/main/Tone_Wajah.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [ ]:
from google.colab import files
import zipfile
import os

# Upload file .zip dari komputer
uploaded = files.upload()
zip_filename = list(uploaded.keys())[0]
print(f"File yang diupload: {zip_filename}")


In [ ]:
extract_dir = './dataset_skintone'

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Cek struktur direktori
for root, dirs, files in os.walk(extract_dir):
    level = root.replace(extract_dir, "").count(os.sep)
    indent = " " * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    for f in files[:3]:
        print(f"{indent}    {f}")

print(f"✅ Ekstrak selesai di: {extract_dir}")


In [ ]:
# Menampilkan jumlah gambar per kelas
train_dir = extract_dir  # Corrected path
class_names = os.listdir(train_dir)
class_counts = {}

print("\nJumlah gambar per kelas:")
for class_name in class_names:
    class_dir = os.path.join(train_dir, class_name)
    # Check if it's a directory before listing files
    if os.path.isdir(class_dir):
        count = len(os.listdir(class_dir))
        class_counts[class_name] = count
        print(f"- {class_name}: {count}")

# Opsional: Menampilkan contoh gambar dari setiap kelas
print("\nContoh gambar dari setiap kelas:")
plt.figure(figsize=(10, 10))
# Only iterate through actual class directories
for i, class_name in enumerate([d for d in class_names if os.path.isdir(os.path.join(train_dir, d))]):
    class_dir = os.path.join(train_dir, class_name)
    sample_image_name = os.listdir(class_dir)[0] # Ambil gambar pertama
    sample_image_path = os.path.join(class_dir, sample_image_name)

    img = load_img(sample_image_path)
    plt.subplot(1, len([d for d in class_names if os.path.isdir(os.path.join(train_dir, d))]), i + 1)
    plt.imshow(img)
    plt.title(class_name)
    plt.axis('off')
plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = datagen.flow_from_directory(
    extract_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    extract_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Menampilkan jumlah gambar di setiap set
print(f"Jumlah gambar training: {train_data.samples}")
print(f"Jumlah gambar validation: {val_data.samples}")


In [ ]:
# Menambahkan augmentasi: rotasi, flipping (horizontal), dan zoom
datagen_augmented = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,        # Rentang rotasi (dalam derajat)
    horizontal_flip=True,     # Mengaktifkan flipping horizontal
    zoom_range=0.2,           # Rentang zoom
    validation_split=0.2      # Tetap menggunakan validasi split
)

# Membuat generator data untuk training dengan augmentasi
train_data_augmented = datagen_augmented.flow_from_directory(
    extract_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Generator data untuk validasi tanpa augmentasi (hanya rescaling)
val_data_augmented = ImageDataGenerator(rescale=1./255, validation_split=0.2).flow_from_directory(
    extract_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Cek contoh batch dari generator yang diaugmentasi
print("\nContoh batch setelah augmentasi:")
img_batch_aug, label_batch_aug = next(iter(train_data_augmented))
print(f"Gambar (augmented): {img_batch_aug.shape}")
print(f"Label (augmented): {label_batch_aug.shape}")

# Menampilkan beberapa contoh gambar yang diaugmentasi (opsional)
print("\nMenampilkan beberapa contoh gambar yang diaugmentasi:")
plt.figure(figsize=(10, 10))
for i in range(9): # Tampilkan 9 gambar
    plt.subplot(3, 3, i + 1)
    plt.imshow(img_batch_aug[i])
    plt.axis('off')
plt.show()

In [ ]:
# prompt: contoh gambar sebelum augmentasi dan sesudah

# Ambil satu contoh gambar dari dataset sebelum augmentasi
class_example = [d for d in class_names if os.path.isdir(os.path.join(train_dir, d))][0] # Ambil kelas pertama
sample_image_path = os.path.join(train_dir, class_example, os.listdir(os.path.join(train_dir, class_example))[0]) # Ambil gambar pertama dari kelas pertama

original_img = load_img(sample_image_path, target_size=(128, 128))
original_img_array = img_to_array(original_img)
# Tambahkan dimensi batch karena flow memerlukan input 4D
original_img_array = np.expand_dims(original_img_array, axis=0)

# Menampilkan gambar asli
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(original_img)
plt.title("Gambar Asli (Sebelum Augmentasi)")
plt.axis('off')

# Menghasilkan beberapa contoh gambar yang diaugmentasi dari gambar asli
augmented_images = []
# Gunakan generator augmentasi yang sudah dibuat
for _ in range(1): # Hanya butuh 1 contoh gambar setelah augmentasi
    for batch in datagen_augmented.flow(original_img_array, batch_size=1):
        augmented_images.append(batch[0])
        break # Keluar setelah mendapatkan 1 gambar

# Menampilkan contoh gambar setelah augmentasi
plt.subplot(1, 2, 2)
# img_to_array mengembalikan float, imshow memerlukan nilai antara 0-1 atau 0-255
# Generator dengan rescale sudah menghasilkan nilai 0-1
plt.imshow(augmented_images[0])
plt.title("Gambar Setelah Augmentasi")
plt.axis('off')

plt.show()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    MaxPooling2D((2,2)),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Definisikan EarlyStopping callback
# pantau val_loss dan hentikan pelatihan jika tidak ada perbaikan selama 5 epoch
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    train_data_augmented,
    validation_data=val_data_augmented,
    epochs=20, # Jumlah epoch bisa lebih besar karena ada early stopping
    callbacks=[early_stopping] # Tambahkan callback early stopping
)

In [ ]:
loss, accuracy = model.evaluate(val_data)
print(f"Akurasi Model: {accuracy * 100:.2f}%")


In [ ]:
  from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import numpy as np
import matplotlib.pyplot as plt

# Get true labels and predictions for the validation data
val_labels = []
val_preds = []
val_data.reset() # Reset generator to start from the beginning

# Iterate through the validation data generator to get all labels and predictions
for _ in range(len(val_data)):
    imgs, labels = next(val_data)
    val_labels.extend(np.argmax(labels, axis=1))
    predictions = model.predict(imgs)
    val_preds.extend(np.argmax(predictions, axis=1))

# Compute the confusion matrix
cm = confusion_matrix(val_labels, val_preds)

# Get class names from the validation generator
class_names = list(val_data.class_indices.keys())

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

# Generate and print the classification report
print("\nClassification Report:")
print(classification_report(val_labels, val_preds, target_names=class_names))


In [ ]:
# Menampilkan grafik training and validation (accuracy and loss)
plt.figure(figsize=(12, 6))

# Plot Accuracy
plt.subplot(1, 2, 1) # 1 row, 2 columns, 1st plot
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim([0, 1]) # Set y-axis limit from 0 to 1 for accuracy

# Plot Loss
plt.subplot(1, 2, 2) # 1 row, 2 columns, 2nd plot
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')

plt.tight_layout() # Adjust layout to prevent overlap
plt.show()


In [ ]:
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
from google.colab import files

# Upload gambar untuk diuji
uploaded = files.upload()
img_path = list(uploaded.keys())[0]

img = load_img(img_path, target_size=(128,128))
img_array = img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

pred = model.predict(img_array)
class_names = list(train_data.class_indices.keys())
print(f"Gambar ini diklasifikasi sebagai: {class_names[np.argmax(pred)]}")


In [ ]:
model.save("cnn_skintone_model.h5")
print("✅ Model berhasil disimpan.")
